In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,  LSTM, Reshape,  TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
!pip install opendatasets

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/dhruvsaluja/indian-medicinal-plants")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dhruvsaluja
Your Kaggle Key: ··········


100%|██████████| 253M/253M [00:04<00:00, 58.4MB/s]


In [3]:
# Define parameters
input_shape = (224, 224, 3)  # Adjust the image size as needed
batch_size = 32
epochs = 10
num_classes = 40  # Number of different plant classes
data_dir = "/content/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset"  # Replace with the path to your dataset folder


In [4]:
# Create a mapping between class indices and plant names
class_mapping = {
    0: 'Aloevera',
    1: 'Amla',
    2: 'Amruta Balli',
    3: 'Arali',
    4: 'Ashoka',
    5: 'Ashwagandha',
    6: 'Avacado',
    7: 'Bamboo',
    8: 'Basale',
    9: 'Betel',
    10: 'Betel_Nut',
    11: 'Brahmi',
    12: 'Castor',
    13: 'Curry Leaf',
    14: 'Doddapatre',
    15: 'Ekka',
    16: 'Ganike',
    17: 'Gauva',
    18: 'Geranium',
    19: 'Henna',
    20: 'Hibiscus',
    21: 'Honge',
    22: 'Insulin',
    23: 'Jasmine',
    24: 'Lemon',
    25: 'Lemon_grass',
    26: 'Mango',
    27: 'Mint',
    28: 'Nagadali',
    29: 'Neem',
    30: 'Nithyapushpa',
    31: 'Nooni',
    32: 'Pappaya',
    33: 'Pepper',
    34: 'Pomegranate',
    35: 'Raktachandini',
    36: 'Rose',
    37: 'Sapota',
    38: 'Tulasi',
    39: 'Wood_sorel',
    # Add mappings for all 40 classes here
}

In [5]:
# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,     # Normalize pixel values
    rotation_range=20,      # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2,        # Shear intensity
    zoom_range=0.2,         # Randomly zoom in on images
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest',    # Fill missing pixels with the nearest value
    validation_split=0.2     # 20% of data will be used for validation
)

In [6]:
# Load and augment training data (80%)
train_generator = datagen.flow_from_directory(
    data_dir,                 # Root directory containing 40 subfolders
    target_size=input_shape[:2],   # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=True,             # Shuffle the data for training
    subset='training'         # Specify training data subset
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset'

In [ ]:
# Load and augment validation data (20%)
validation_generator = datagen.flow_from_directory(
    data_dir,                 # Root directory containing 40 subfolders
    target_size=input_shape[:2],  # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False,            # Do not shuffle for validation
    subset='validation'       # Specify validation data subset
)


In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier
from keras.optimizers import Adam

# Number of models in the ensemble
num_models = 3
models = []

# Create and train individual models
for _ in range(num_models):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
    )

    models.append(model)

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from keras.optimizers import Adam
import seaborn as sns
import matplotlib.pyplot as plt

# Save each individual model
for i, model in enumerate(models):
    model.save(f'plant_classification_model_{i}.h5')

# Make predictions on each individual model
val_predictions = np.array([model.predict(validation_generator) for model in models])
val_predictions = np.mean(val_predictions, axis=0)
val_predictions_classes = np.argmax(val_predictions, axis=-1)
val_true_classes = validation_generator.classes

# Calculate metrics
accuracy = accuracy_score(val_true_classes, val_predictions_classes)
precision = precision_score(val_true_classes, val_predictions_classes, average='weighted')
recall = recall_score(val_true_classes, val_predictions_classes, average='weighted')
f1 = f1_score(val_true_classes, val_predictions_classes, average='weighted')
classification_report_str = classification_report(val_true_classes, val_predictions_classes, target_names=class_mapping.values())
conf_matrix = confusion_matrix(val_true_classes, val_predictions_classes)

print(f"Ensemble Model Metrics:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Classification Report:")
print(classification_report_str)
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_mapping.values(), yticklabels=class_mapping.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from keras.preprocessing import image

# Example code for prediction on a new image using the ensemble model
def predict_plant_ensemble(image_path, models):
    img = image.load_img(image_path, target_size=input_shape[:2])
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0

    # Make predictions on each individual model
    model_predictions = np.array([model.predict(img) for model in models])
    ensemble_prediction = np.mean(model_predictions, axis=0)

    class_index = np.argmax(ensemble_prediction)
    plant_name = class_mapping.get(class_index, 'Unknown Plant')
    confidence = np.max(ensemble_prediction)

    return plant_name, confidence

# Example usage:
image_path = '/content/aloe.jpg'
predicted_plant, confidence = predict_plant_ensemble(image_path, models)
print(f"Predicted plant: {predicted_plant}")
print(f"Confidence level: {confidence * 100:.2f}%")


In [ ]:
# Example code for prediction on a new image using the ensemble model
def predict_plant_ensemble(image_path):
    img = image.load_img(image_path, target_size=input_shape[:2])
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0

    ensemble_prediction = loaded_ensemble_model.predict(img)
    class_index = np.argmax(ensemble_prediction)
    plant_name = class_mapping.get(class_index, 'Unknown Plant')
    confidence = np.max(ensemble_prediction)

    return plant_name, confidence

In [ ]:
# Example usage:
image_path = '/content/aloe.jpg'
predicted_plant, confidence = predict_plant_ensemble(image_path)
print(f"Predicted plant: {predicted_plant}")
print(f"Confidence level: {confidence * 100:.2f}%")